In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
import matplotlib.pyplot as plt

# ================================
#  Device Setup
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
torch.backends.cudnn.benchmark = True

# ================================
#  Data Transformations
# ================================
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ================================
#  Load Lung Cancer Dataset
# ================================
test_dir = r'/content/drive/MyDrive/Data/Val'  # <-- Your validation/test set

if not os.path.exists(test_dir):
    raise FileNotFoundError("Test directory not found. Check the path!")

test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print("Class Mapping:", test_dataset.class_to_idx)  # e.g., {'benign': 0, 'malignant': 1}

# ================================
#  Define ViT Model for Zero-Shot
# ================================
class ViTCancerClassifier(nn.Module):
    def __init__(self):
        super(ViTCancerClassifier, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)

        # Freeze all pretrained ViT layers (zero-shot)
        for param in self.vit.parameters():
            param.requires_grad = False

        # Add classification head (can be optionally trained later)
        in_features = self.vit.head.in_features
        self.vit.head = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)  # Binary classification output
        )

    def forward(self, x):
        return self.vit(x)

# Initialize model
model = ViTCancerClassifier().to(device)

# Loss (only used for evaluation)
criterion = nn.BCEWithLogitsLoss()

# ================================
#  Evaluation Function (Zero-Shot)
# ================================
def evaluate(model, test_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    print(f"Zero-Shot Test Loss: {total_loss / len(test_loader):.4f}")
    print(f"Zero-Shot Test Accuracy: {accuracy:.4f}")

# ================================
#  Run Zero-Shot Inference
# ================================
if __name__ == '__main__':
    evaluate(model, test_loader, criterion, device)


Using device: cuda
Class Mapping: {'cancer': 0, 'normal lung tissue': 1}
Zero-Shot Test Loss: 0.7056
Zero-Shot Test Accuracy: 0.4500


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import timm
import random

# ================================
#  Check Device
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.backends.cudnn.benchmark = True

# ================================
#  Data Preprocessing & Augmentation
# ================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(degrees=0, shear=10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ================================
#  Load Datasets (Check Paths)
# ================================
train_dir = r'/content/drive/MyDrive/Data/Train'
test_dir = r'/content/drive/MyDrive/Data/Val'

if not os.path.exists(train_dir) or not os.path.exists(test_dir):
    raise FileNotFoundError("Dataset directories not found. Check the paths!")

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# One shot learning
num_samples = int(0.005 * len(train_dataset))
indices = random.sample(range(len(train_dataset)), num_samples)
train_subset = Subset(train_dataset, indices)
print(num_samples)

# Data Loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print("Class Mapping:", train_dataset.class_to_idx)

# ================================
#  Define Pretrained ViT Model for Binary Classification
# ================================
class ViTCancerClassifier(nn.Module):
    def __init__(self):
        super(ViTCancerClassifier, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        in_features = self.vit.head.in_features
        self.vit.head = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.vit(x)

# Create Model
model = ViTCancerClassifier().to(device)

# Define Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ================================
# Training Loop
# ================================
import matplotlib.pyplot as plt

def train(model, train_loader, criterion, optimizer, device, epochs=5):
    model.train()
    epoch_accuracies = []

    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

        accuracy = correct / total
        epoch_accuracies.append(accuracy)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}")


# ================================
# Evaluation Loop
# ================================
def evaluate(model, test_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    print(f"Test Loss: {total_loss/len(test_loader):.4f}, Test Accuracy: {correct/total:.4f}")

# ================================
#  Run Training & Evaluation
# ================================
if __name__ == '__main__':
    train(model, train_loader, criterion, optimizer, device, epochs=10)
    evaluate(model, test_loader, criterion, device)


Using device: cuda
1
Class Mapping: {'cancer': 0, 'normal lung tissue': 1}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Epoch [1/10], Loss: 0.5984, Accuracy: 1.0000
Epoch [2/10], Loss: 0.5494, Accuracy: 1.0000
Epoch [3/10], Loss: 0.0558, Accuracy: 1.0000
Epoch [4/10], Loss: 0.0251, Accuracy: 1.0000
Epoch [5/10], Loss: 0.0178, Accuracy: 1.0000
Epoch [6/10], Loss: 0.0013, Accuracy: 1.0000
Epoch [7/10], Loss: 0.0011, Accuracy: 1.0000
Epoch [8/10], Loss: 0.0006, Accuracy: 1.0000
Epoch [9/10], Loss: 0.0038, Accuracy: 1.0000
Epoch [10/10], Loss: 0.0004, Accuracy: 1.0000
Test Loss: 4.0795, Test Accuracy: 0.5000


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import timm
import random

# ================================
#  Check Device
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.backends.cudnn.benchmark = True

# ================================
#  Data Preprocessing & Augmentation
# ================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(degrees=0, shear=10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ================================
#  Load Datasets (Check Paths)
# ================================
train_dir = r'/content/drive/MyDrive/Data/Train'
test_dir = r'/content/drive/MyDrive/Data/Val'

if not os.path.exists(train_dir) or not os.path.exists(test_dir):
    raise FileNotFoundError("Dataset directories not found. Check the paths!")

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# One shot learning
num_samples = int(0.005 * len(train_dataset))
indices = random.sample(range(len(train_dataset)), num_samples)
train_subset = Subset(train_dataset, indices)
print(num_samples)

# Data Loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print("Class Mapping:", train_dataset.class_to_idx)

# ================================
#  Define Pretrained ViT Model for Binary Classification
# ================================
class ViTCancerClassifier(nn.Module):
    def __init__(self):
        super(ViTCancerClassifier, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        in_features = self.vit.head.in_features
        self.vit.head = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.vit(x)

# Create Model
model = ViTCancerClassifier().to(device)

# Define Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ================================
# Training Loop
# ================================
import matplotlib.pyplot as plt

def train(model, train_loader, criterion, optimizer, device, epochs=5):
    model.train()
    epoch_accuracies = []

    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

        accuracy = correct / total
        epoch_accuracies.append(accuracy)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}")


# ================================
# Evaluation Loop
# ================================
def evaluate(model, test_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    print(f"Test Loss: {total_loss/len(test_loader):.4f}, Test Accuracy: {correct/total:.4f}")

# ================================
#  Run Training & Evaluation
# ================================
if __name__ == '__main__':
    train(model, train_loader, criterion, optimizer, device, epochs=5)
    evaluate(model, test_loader, criterion, device)


Using device: cuda
1
Class Mapping: {'cancer': 0, 'normal lung tissue': 1}
Epoch [1/5], Loss: 0.5134, Accuracy: 1.0000
Epoch [2/5], Loss: 0.1095, Accuracy: 1.0000
Epoch [3/5], Loss: 0.0225, Accuracy: 1.0000
Epoch [4/5], Loss: 0.0023, Accuracy: 1.0000
Epoch [5/5], Loss: 0.0006, Accuracy: 1.0000
Test Loss: 3.5978, Test Accuracy: 0.5000


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import timm
import random

# ================================
#  Check Device
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.backends.cudnn.benchmark = True

# ================================
#  Data Preprocessing & Augmentation
# ================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(degrees=0, shear=10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ================================
#  Load Datasets (Check Paths)
# ================================
train_dir = r'/content/drive/MyDrive/Data/Train'
test_dir = r'/content/drive/MyDrive/Data/Val'

if not os.path.exists(train_dir) or not os.path.exists(test_dir):
    raise FileNotFoundError("Dataset directories not found. Check the paths!")

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# Five-shot learning
num_samples = int(0.025 * len(train_dataset))
indices = random.sample(range(len(train_dataset)), num_samples)
train_subset = Subset(train_dataset, indices)
print(num_samples)

# Data Loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print("Class Mapping:", train_dataset.class_to_idx)

# ================================
#  Define Pretrained ViT Model for Binary Classification
# ================================
class ViTCancerClassifier(nn.Module):
    def __init__(self):
        super(ViTCancerClassifier, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        in_features = self.vit.head.in_features
        self.vit.head = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.vit(x)

# Create Model
model = ViTCancerClassifier().to(device)

# Define Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ================================
# Training Loop
# ================================
import matplotlib.pyplot as plt

def train(model, train_loader, criterion, optimizer, device, epochs=5):
    model.train()
    epoch_accuracies = []

    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

        accuracy = correct / total
        epoch_accuracies.append(accuracy)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}")



# ================================
# Evaluation Loop
# ================================
def evaluate(model, test_loader, criterion, device):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, roc_auc_score

    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0

    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)

            probs = torch.sigmoid(outputs)
            preds = probs > 0.5

            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    print(f"Test Loss: {total_loss/len(test_loader):.4f}, Test Accuracy: {correct/total:.4f}")




# ================================
#  Run Training & Evaluation
# ================================
if __name__ == '__main__':
    train(model, train_loader, criterion, optimizer, device, epochs=5)
    evaluate(model, test_loader, criterion, device)


Using device: cuda
5
Class Mapping: {'cancer': 0, 'normal lung tissue': 1}
Epoch [1/5], Loss: 0.6532, Accuracy: 1.0000
Epoch [2/5], Loss: 0.6362, Accuracy: 0.6000
Epoch [3/5], Loss: 0.5275, Accuracy: 0.8000
Epoch [4/5], Loss: 0.8813, Accuracy: 0.4000
Epoch [5/5], Loss: 0.3645, Accuracy: 0.8000
Test Loss: 0.5185, Test Accuracy: 0.5167


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import timm
import random

# ================================
#  Check Device
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.backends.cudnn.benchmark = True

# ================================
#  Data Preprocessing & Augmentation
# ================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(degrees=0, shear=10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ================================
#  Load Datasets (Check Paths)
# ================================
train_dir = r'/content/drive/MyDrive/Data/Train'
test_dir = r'/content/drive/MyDrive/Data/Val'

if not os.path.exists(train_dir) or not os.path.exists(test_dir):
    raise FileNotFoundError("Dataset directories not found. Check the paths!")

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# Reduce training dataset to 5% (Dataset Partioning)

num_samples = int(0.05 * len(train_dataset))
indices = random.sample(range(len(train_dataset)), num_samples)
train_subset = Subset(train_dataset, indices)
print(num_samples)

# Data Loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print("Class Mapping:", train_dataset.class_to_idx)

# ================================
#  Define Pretrained ViT Model for Binary Classification
# ================================
class ViTCancerClassifier(nn.Module):
    def __init__(self):
        super(ViTCancerClassifier, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        in_features = self.vit.head.in_features
        self.vit.head = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.vit(x)

# Create Model

model = ViTCancerClassifier().to(device)

# Define Loss and Optimizer

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


# ================================
# Training Loop
# ================================
import matplotlib.pyplot as plt

def train(model, train_loader, criterion, optimizer, device, epochs=5):
    model.train()
    epoch_accuracies = []

    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

        accuracy = correct / total
        epoch_accuracies.append(accuracy)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}")



# ================================
# Evaluation Loop
# ================================

def evaluate(model, test_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    print(f"Test Loss: {total_loss/len(test_loader):.4f}, Test Accuracy: {correct/total:.4f}")


# ================================
#  Run Training & Evaluation
# ================================

if __name__ == '__main__':
    train(model, train_loader, criterion, optimizer, device, epochs=10)
    evaluate(model, test_loader, criterion, device)


Using device: cuda
11
Class Mapping: {'cancer': 0, 'normal lung tissue': 1}
Epoch [1/10], Loss: 0.7906, Accuracy: 0.4545
Epoch [2/10], Loss: 0.6261, Accuracy: 0.6364
Epoch [3/10], Loss: 0.4557, Accuracy: 0.8182
Epoch [4/10], Loss: 0.4897, Accuracy: 0.8182
Epoch [5/10], Loss: 0.3140, Accuracy: 0.9091
Epoch [6/10], Loss: 0.8181, Accuracy: 0.5455
Epoch [7/10], Loss: 0.1895, Accuracy: 1.0000
Epoch [8/10], Loss: 0.2130, Accuracy: 0.9091
Epoch [9/10], Loss: 0.0996, Accuracy: 1.0000
Epoch [10/10], Loss: 0.0208, Accuracy: 1.0000
Test Loss: 0.0870, Test Accuracy: 0.9667
